<a href="https://colab.research.google.com/github/matteo-ticli/cars_detection_COCOdataset/blob/main/cars_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
## install detectron2: (Colab has CUDA 10.1 + torch 1.8)
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

In [ ]:
## Connect to MyDrive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
path = '/content/drive/MyDrive/datrix/'

## Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

## import some common libraries
import numpy as np
import pandas as pd
import os, json, cv2, random
from google.colab.patches import cv2_imshow

## import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
## list all file_name in folder "cars"

def list_images(folder):
    images = []
    for filename in os.listdir(folder):
        if filename[-4:] == '.jpg':
          images.append(filename)
    return images

In [ ]:
## run "mask_rcnn_R_50_FPN_3x" on the entire dataset
## save bbox for each file_name in a dataframe and the associated probabilities to infer conclusions on the model

def get_obj(label, folder = path+"/cars", show_pred = False):

    images = list_images(folder)
    filename_list = list()
    bbox_list = list()
    scores_list = list()

    for pic in images:
        im = cv2.imread(folder+'/'+pic)
        ## build "mask_rcnn_R_50_FPN_3x"
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        predictor = DefaultPredictor(cfg)
        outputs = predictor(im)

        ## set show_pred to True in order to show predictions on the images
        if show_pred == True:
            v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            cv2_imshow(out.get_image()[:, :, ::-1])
        
        ## we need to save the position of the label "car" i.e. "2" because we store the pred_boxes of the cars
        ## we know that the label "car" is represented by the number "2" from an initial inspection of the dataset
        pos_items = [i for i,x in enumerate(outputs['instances'].pred_classes) if x == label]
  
        Box = outputs["instances"].pred_boxes
        Box_list = list(Box.tensor.cpu().numpy())
        bound_boxes = [Box_list[i] for i in pos_items]
        
        Scores = outputs["instances"].scores
        Scores_list = Scores.cpu().numpy()
        scores = [Scores_list[i] for i in pos_items]
        
        ## fill containers that will be used as columns of dataframe
        for j in range(len(bound_boxes)):
            filename_list.append(pic)
            bbox_list.append(bound_boxes[j])
            scores_list.append(scores[j])
    
    df = {'file_name': filename_list, 'bounding_box': bbox_list}
    DF = pd.DataFrame(data=df)
    DF = DF.sort_values(by=['file_name'])

    ## evaluation of the scores
    l = len(scores_list)
    S = np.sum(scores_list)
    avg = S/l
    print(f'The average probability of true detection is: {avg}')

    return DF

In [ ]:
## we are going to use the label 2 as it is the label used to identify cars in "mask_rcnn_R_50_FPN_3x"
DF = get_obj(2)

In [ ]:
DF.to_csv(path+'/final_output.csv', index=False)

To access cars in the provided images, I have deployed Facebook Detectron2 open-source library, which provides several models for image detection. I have used the "mask_rcnn_R_50_FPN_3x" model to make predictions on the images. Given my little experience in this field, I have tried a basic model which grants a decent average probability (0.84) of correct detection for the 800 objects detected. Further analysis, could be conducted on two main areas: perform detection with other models and change the threshold model.